# ML functions

In [1]:
import torch

## Train

In [2]:
def train(net, criterion, optimizer, dataloader, params: dict = {}):
    if 'epoch' not in params:
        params['epoch'] = 0
    if 'cuda' not in params:
        params['cuda'] = False
    if 'log' not in params:
        params['log'] = False
    if 'logFile' not in params:
        if params['log']:
            params['logFile'] = open('train.csv', 'w')
        else:
            params['logFile'] = None

    epoch, cuda, log, logFile = params['epoch'], params['cuda'], params['log'], params['logFile']

    net.train()  # tells net to do training

    # for log
    nProcessed = 0
    nTrain = len(dataloader.dataset)

    for batch_idx, (inputs, targets) in enumerate(dataloader):
        if cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        # sets gradient to 0
        optimizer.zero_grad()

        # forward, backward, and opt
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        # log
        nProcessed += len(inputs)
        pred = outputs.data.max(dim=1)[1]  # get the index of the max log-probability
        incorrect = pred.ne(targets.data).cpu().sum()  # ne: not equal
        err = 100. * incorrect / len(inputs)
        partialEpoch = params['epoch'] + batch_idx / len(dataloader)

        if log and (logFile is not None):  # saves at csv file
            logFile.write('{},{},{}\n'.format(partialEpoch, loss.item(), err))
            logFile.flush()

        else:  # print at STDOUT
            print('Train Epoch: {:.2f} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tError: {:.6f}'.format(
                partialEpoch, nProcessed, nTrain, 100. * batch_idx / len(dataloader), loss.item(), err
            ), end='\r')

## Test

In [3]:
def test(net, criterion, dataloader, params: dict = {}):
    if 'epoch' not in params:
        params['epoch'] = 0
    if 'cuda' not in params:
        params['cuda'] = False
    if 'log' not in params:
        params['log'] = False
    if 'logFile' not in params:
        if params['log']:
            params['logFile'] = open('test.csv', 'w')
        else:
            params['logFile'] = None

    epoch, cuda, log, logFile = params['epoch'], params['cuda'], params['log'], params['logFile']

    net.eval()  # tells net to do evaluating

    # for log
    test_loss = 0.
    incorrect = 0.

    for inputs, targets in dataloader:
        if cuda:
            inputs, targets = inputs.cuda(), targets.cuda()

        # eval
        with torch.no_grad():
            outputs = net(inputs)
            test_loss += criterion(outputs, targets).item()
            pred = outputs.data.max(dim=1)[1]  # get the index of the max log-probability
            incorrect += pred.ne(targets.data).cpu().sum()  # ne: not equal

    # log
    test_loss /= len(dataloader)  # loss function already averages over batch size
    nTotal = len(dataloader.dataset)
    err = 100. * incorrect / nTotal

    if log and (logFile is not None):  # saves at csv file
        logFile.write('{},{},{}\n'.format(epoch, test_loss, err))
        logFile.flush()
    else:  # print at STDOUT
        print('\nTest Set\tAverage Loss: {:.4f}\tError: {}/{} ({:.06f}%)\n'.format(
            test_loss, incorrect, nTotal, err
        ))